# Lese web_logs som Canvas_history
Først importerer eg alle nødvendige funksjonar:

In [3]:
import akvut
import pandas as pd


Så les eg `web_logs` frå det siste døgeret (det tok 6'47.2''):

In [2]:

logger = akvut.lag_logger('timer.log')
data = akvut.les_web_logs(logger)
print(data.info())


2025-02-12 07:57:59,596 - my_logger - INFO - Henta access_token OK.
2025-02-12 07:57:59,611 - my_logger - INFO - payload er {"format": "csv", "since": "2025-02-11T07:57:59.611654Z"}
2025-02-12 07:57:59,613 - my_logger - INFO - Sender (inkrementel) spørjing til https://api-gateway.instructure.com/dap/query/canvas_logs/table/web_logs/data


Har ikkje henta frå Azure


2025-02-12 07:58:00,794 - my_logger - INFO - Sjekker status på jobb f397d8ba-ed09-4926-896c-62650f02f488
2025-02-12 07:58:06,340 - my_logger - INFO - Sjekker status på jobb f397d8ba-ed09-4926-896c-62650f02f488
2025-02-12 07:58:11,756 - my_logger - INFO - Sjekker status på jobb f397d8ba-ed09-4926-896c-62650f02f488
2025-02-12 07:58:17,214 - my_logger - INFO - Sjekker status på jobb f397d8ba-ed09-4926-896c-62650f02f488
2025-02-12 07:58:22,634 - my_logger - INFO - Sjekker status på jobb f397d8ba-ed09-4926-896c-62650f02f488
2025-02-12 07:58:28,033 - my_logger - INFO - Sjekker status på jobb f397d8ba-ed09-4926-896c-62650f02f488
2025-02-12 07:58:33,435 - my_logger - INFO - Sjekker status på jobb f397d8ba-ed09-4926-896c-62650f02f488
2025-02-12 07:58:38,830 - my_logger - INFO - Sjekker status på jobb f397d8ba-ed09-4926-896c-62650f02f488
2025-02-12 07:58:44,211 - my_logger - INFO - Sjekker status på jobb f397d8ba-ed09-4926-896c-62650f02f488
2025-02-12 07:58:49,615 - my_logger - INFO - Sjekker st

Feil ved opplasting av sist oppdatert: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect); [IM002] [Microsoft][ODBC Driver Manager] Invalid connection string attribute (0)')
<class 'pandas.core.frame.DataFrame'>
Index: 2716428 entries, 0 to 1515240
Data columns (total 25 columns):
 #   Column                              Dtype 
---  ------                              ----- 
 0   key.id                              object
 1   value.timestamp                     object
 2   value.user_id                       object
 3   value.real_user_id                  object
 4   value.course_id                     object
 5   value.quiz_id                       object
 6   value.discussion_id                 object
 7   value.conversation_id               object
 8   value.assignment_id                 object
 9   value.url                           object
 10  value.http_method                   object
 11  value.

Når eg les `web_logs` lagrer eg rådata i ei fil (`ny_history_alle.csv`). Men eg kan no sende desse vidare til nye rutiner der eg henter ut andre typer data.

In [ ]:

akvut.ny_history(data, logger)

## Alias
Eg prøver å lage statistikk over kven som opptrer som kven. Her har eg `user_id` på dei mest vanlege superbrukarane, og så kan eg lage ei liste over 

- kven dei har opptrådd som
- kor mange ganger dei har opptredd

NB! Det er mange av desse som er "Studentvising", slik at det er mange "user_id" som ikkje finst.


In [4]:
alias = data[~data['value.real_user_id'].isnull()]
adm_namn = {'2916': 'Aasmund', '12353': 'Johan', '3861': 'Casper', '3990': 'Dag Ove', '26737': 'Hege', '97393': 'Henrik', '35965': 'Jan Erik', '14186': 'Andreas Senneset Hove', '6': 'Ivar', '1703': 'Terje'}
adm_liste = []
for adm in ["1703", "2916", "12353", "97393", "3861", "14186", "35965", "98283", "5036", "6", "86785", "26737", "3990"]:
    adm_liste.append({'user': adm, 'count': alias[alias['value.real_user_id'] == adm]['value.user_id'].value_counts()})
for a in adm_liste:
    if len(a['count']) > 0:
        print(f"Administrator: {a['user']} ({adm_namn[a['user']]})")
        for index, item in a['count'].items():
            print(f"{index}: {item}")


Administrator: 12353 (Johan)
12353: 99
4552: 81
110857: 48
111161: 28


## Data (til history)
Så til det eg eigentleg skulle gjere: hente ut data til U4-prosjektet. Det er tre krav:
1. Feltet `user_id` må innehalde noko
2. Feltet `value.http_status` skal være `200`
3. Feltet `value.url` skal ikkje starte på "/api/"

In [5]:
utval = data[(data["value.user_id"].notnull()) & (data["value.http_status"].str.startswith("2")) & (~data["value.url"].str.startswith("/api/"))]


Det er litt usikkert kva vi skal bruke det til, men eg kan sende data til Håvard og be han plukke ut. 

In [6]:
utval.to_csv("history_250211_alle.csv")

Eg kan kanskje sende berre dei som er interessante (håper eg treff på utvalet):

In [7]:
redusert_utval = utval[['value.timestamp', 'value.user_id', 'value.course_id', 'value.url', 'value.web_application_controller', 'value.web_application_action', 'value.web_application_context_type']]


In [9]:
redusert_utval['dato'] = redusert_utval['value.timestamp'].apply(lambda x: x[0:10])

C:\Users\AKV\AppData\Local\Temp\ipykernel_22788\2458858629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  redusert_utval['dato'] = redusert_utval['value.timestamp'].apply(lambda x: x[0:10])


In [10]:
redusert_utval.to_csv("history_250210_redusert_utval.csv")

Her er ei linje for å hente data inn att frå fil:

In [4]:
redusert_utval = pd.read_csv("history_250210_redusert_utval.csv")

Eg kan lage ei liste over kor mange aktive hendingar det er per student:

In [7]:
redusert_utval['value.user_id'].value_counts()

value.user_id
106979    606
109608    569
4945      458
108737    421
92781     393
         ... 
51100       1
110034      1
83652       1
3           1
37711       1
Name: count, Length: 12048, dtype: int64

In [8]:
redusert_utval

,Unnamed: 0,value.timestamp,value.user_id,value.course_id,value.url,value.web_application_controller,value.web_application_action,value.web_application_context_type,dato
0,3,2025-02-11T11:00:16.483Z,72905,28317.0,/login/session_token?return_to=https://hvl.ins...,login,session_token,NaN,2025-02-11
1,4,2025-02-11T11:00:16.730Z,107264,NaN,/web-app-manifest/manifest.json,info,web_app_manifest,NaN,2025-02-11
2,18,2025-02-11T11:00:18.538Z,70077,NaN,/page_views/e52bb05e-b868-465c-8e75-cc47a10344...,page_views,update,NaN,2025-02-11
3,23,2025-02-11T11:00:18.747Z,94276,28464.0,/courses/28464,courses,show,Course,2025-02-11
4,32,2025-02-11T11:00:19.432Z,102716,NaN,/,users,user_dashboard,User,2025-02-11
...,...,...,...,...,...,...,...,...,...
342966,1515186,2025-02-12T04:04:18.285Z,93018,NaN,/login/session_token?no_verifiers=1,login,session_token,NaN,2025-02-12
342967,1515211,2025-02-12T04:04:22.301Z,70444,NaN,/feeds/calendars/user_slJ0Z4kUtYew75rhHHnPXFYM...,calendar_events_api,public_feed,User,2025-02-12
342968,1515234,2025-02-12T04:04:38.152Z,83548,NaN,/feeds/calendars/user_jMF1k8IeTdaB2yJImv34t5mn...,calendar_events_api,public_feed,User,2025-02-12
342969,1515235,2025-02-12T04:04:39.159Z,79848,NaN,/feeds/calendars/user_IAiIGrYNo9U9w9RT8b4HBomQ...,calendar_events_api,public_feed,User,2025-02-12


## Leit etter duplikat
Det neste vert å leite etter duplikat. Det vil sei: rader som har lik verdi i fleire celler. Då må eg eigentleg lage ei ny dataramme; eg går gjennom den første og finn like rader, og så legg eg denne raden (og antalet) i ei ny ramme. Vi prøver med noko enkelt:

`user_id`, `course_id`, `web_application_controller`, `web_application_action` og `dato` er like.


In [9]:
temp_duplikat = redusert_utval[["value.user_id", "value.course_id", "value.web_application_controller", "value.web_application_action", "value.web_application_context_type", "dato"]].value_counts()
duplikat = temp_duplikat.to_frame('frekvens').reset_index()

Og så lagrer eg til fil før eg ser på detaljane:

In [14]:
duplikat.to_csv('duplikat_250211.csv')

Først er eg på dei fem mest brukte linjene:

In [12]:
duplikat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55952 entries, 0 to 55951
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   value.user_id                       55952 non-null  int64  
 1   value.course_id                     55952 non-null  float64
 2   value.web_application_controller    55952 non-null  object 
 3   value.web_application_action        55952 non-null  object 
 4   value.web_application_context_type  55952 non-null  object 
 5   dato                                55952 non-null  object 
 6   frekvens                            55952 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 3.0+ MB


Her har altså student 106979 gjort same kommando 265 ganger! For å finne ut kva det er må eg nesten sjekke med rådata (av di dette er ein quiz har eg ikkje tatt med data om quizen i utvalet, og eg må drive litt detektivarbeid for å finne kva quiz det er):

In [16]:
student_106979 = data[(data['value.user_id'] == "106979") & (data['value.quiz_id'] == "18705")]
student_106979.to_csv('student_106979.csv', index=False)

Så tar eg ut ei liste over kor mange ganger kva duplikat-rad opptrer; det viser seg jo at dei aller, aller fleste radene er unike:

In [11]:
duplikat['frekvens'].value_counts().sort_index()

frekvens
1      32004
2      10816
3       4854
4       2752
5       1651
       ...  
154        1
169        1
176        1
236        1
265        1
Name: count, Length: 101, dtype: int64

In [18]:
temp99 = redusert_utval[redusert_utval["value.web_application_controller"]=="wiki_pages"]
håvard = temp99[["value.user_id", "value.course_id", "value.url"]].value_counts()
håvard_ramme = håvard.to_frame('frekvens').reset_index()

In [20]:
håvard_ramme.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6052 entries, 0 to 6051
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   value.user_id    6052 non-null   int64  
 1   value.course_id  6052 non-null   float64
 2   value.url        6052 non-null   object 
 3   frekvens         6052 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 189.3+ KB


In [24]:
håvard_ramme[håvard_ramme['value.user_id']==4485]

,value.user_id,value.course_id,value.url,frekvens
53,4485,14798.0,/courses/14798/pages/brukerkunnskap-og-brukerm...,7
54,4485,14798.0,/courses/14798/pages/brukerkunnskap-og-brukerm...,7
560,4485,14798.0,/courses/14798/pages/profesjonsspesifikke-ekse...,3
697,4485,14798.0,/courses/14798/pages/modellen-for-kunnskapsbas...,3
698,4485,14798.0,/courses/14798/pages/modellen-for-kunnskapsbas...,3
1479,4485,14798.0,/courses/14798/pages/profesjonsspesifikke-ekse...,2
1480,4485,14798.0,/courses/14798/pages/profesjonsspesifikke-ekse...,2
1482,4485,14798.0,/courses/14798/pages/reflekter-over-styrker-og...,2
1487,4485,14798.0,/courses/14798/pages/trinnene-i-kunnskapsbaser...,2
1491,4485,14798.0,/courses/14798/pages/yrkesetiske-retningslinje...,2
